### AUTH

In [1]:
# Cloud authentication.
from google.colab import auth
auth.authenticate_user()

In [2]:
# Import, authenticate and initialize the Earth Engine library.
import ee
ee.Authenticate()
ee.Initialize(project='ee-USERNAME/PROJECT')

### **Geometries**

In [3]:
import ee
import folium
from IPython.display import HTML

#Initialize the Earth Engine API
ee.Initialize()

#Define the study area
geometryPINEGULCH = ee.Geometry.Polygon(
        [[[-108.80164742558028, 39.28402267340665],
          [-108.23722481815841, 39.29358857295844],
          [-108.23997140018966, 39.56831366828538],
          [-108.79752755253341, 39.553491364366145]
          ]])

geometryGRIZZLYCREEK = ee.Geometry.Polygon(
        [[[-107.40066503796388, 39.69746161256133],
          [-107.39860510144044, 39.46619798632684],
          [-106.90834020886231, 39.46778823854969],
          [-106.96670507702638, 39.74763415825682],
          [-107.40272497448731, 39.73337830588799]
          ]])

geometryCHERRYCANYON = ee.Geometry.Polygon(
        [[[-103.53011965523325, 37.51499254964799],
          [-103.53011965523325, 37.336135915833296],
          [-103.33785891304575, 37.33176822282621],
          [-103.33785891304575, 37.519349567331446]]]);

geometrySILVERCREEK = ee.Geometry.Polygon(
        [[[-105.29672060223413, 37.6911297494477],
          [-105.29672060223413, 37.35348518722596],
          [-104.96850404949976, 37.35348518722596],
          [-104.96850404949976, 37.6911297494477]]]);
# 2017-07-19

geometryDECKER = ee.Geometry.Polygon(
        [[[-106.02963654395263, 38.479133428395656],
          [-106.02963654395263, 38.38392784402862],
          [-105.91977326270263, 38.38392784402862],
          [-105.91977326270263, 38.479133428395656]]]);

# 2019-09-08

geometryLAKECHRIST = ee.Geometry.Polygon(
        [[[-107.09745152809535, 39.473198422531205],
          [-107.09745152809535, 39.36445604809376],
          [-106.97797520973597, 39.36445604809376],
          [-106.97797520973597, 39.473198422531205]]]);
#2018-07-30

geometryPALATEAU = ee.Geometry.Polygon(
        [[[-108.55952873183622, 37.67620981411429],
          [-108.55952873183622, 37.534774582410634],
          [-108.39198722792997, 37.534774582410634],
          [-108.39198722792997, 37.67620981411429]]]);


geometries_dict = {
    'PineGulch': geometryPINEGULCH,
    'GrizzlyCreek': geometryGRIZZLYCREEK,
    'CherryCanyon': geometryCHERRYCANYON,
    'SilverCreek': geometrySILVERCREEK,
    'Decker': geometryDECKER,
    'LakeChrist': geometryLAKECHRIST,
    'Palateau': geometryPALATEAU
}

geometries = [
    geometryPINEGULCH,
    geometryGRIZZLYCREEK,
    geometryCHERRYCANYON,
    geometrySILVERCREEK,
    geometryDECKER,
    geometryLAKECHRIST,
    geometryPALATEAU
]

In [ ]:
def get_fire_boundary(event_id):
    dataset = ee.FeatureCollection('USFS/GTAC/MTBS/burned_area_boundaries/v1')
    filtered_dataset = dataset.filter(ee.Filter.eq('Event_ID', event_id))

    if filtered_dataset.size().getInfo() > 0:
        selected_fire = ee.Feature(filtered_dataset.first())
        fire_geometry = selected_fire.geometry()
        geometry_type = fire_geometry.type().getInfo()

        if geometry_type == 'Polygon':
            coordinates = fire_geometry.coordinates().getInfo()
            fire_polygon = ee.Geometry.Polygon(coordinates)
            return fire_polygon
        elif geometry_type == 'GeometryCollection':
            geometries = fire_geometry.geometries().getInfo()
            fire_multipolygon = ee.Geometry.MultiPolygon(geometries)
            return fire_multipolygon
        else:
            print(f"Unsupported geometry type: {geometry_type}")
            return None
    else:
        print(f"No features found with the specified Event_ID: {event_id}")
        return None


event_id_PineGulch = 'CO3933610852620200731'
event_id_GrizzlyCreek = 'CO3957210726620200810'
event_id_CherryCanyon = 'CO3736710345020200520'
event_id_Decker = 'CO3840910600420190908'
event_id_LakeChrist = 'CO3937110704320180703'
event_id_Palateau = 'CO3765810847420180722'
event_id_SilverCreek = 'CO4022310665520180719'


event_id_dict = {
    'PineGulch': 'CO3933610852620200731',
    'GrizzlyCreek': 'CO3957210726620200810',
    'CherryCanyon': 'CO3736710345020200520',
    'Decker': 'CO3840910600420190908',
    'LakeChrist': 'CO3937110704320180703',
    'Palateau': 'CO3765810847420180722',
    'SilverCreek': 'CO4022310665520180719'
}

fire_boundaries = {}

#for fire_name, event_id in event_id_dict.items():
#    fire_boundary = get_fire_boundary(event_id)
#
#    if fire_boundary is not None:
#        fire_boundaries[fire_name] = fire_boundary
#        print(f"Added fire boundary for {fire_name} to the fire_boundaries dictionary:")
#        print(fire_boundaries[fire_name].getInfo())

for fire_name, polygon in geometries_dict.items():
    #fire_boundary = get_fire_boundary(polygon)

    if polygon is not None:
        fire_boundaries[fire_name] = polygon
        print(f"Added fire boundary for {fire_name} to the fire_boundaries dictionary:")
        print(fire_boundaries[fire_name].getInfo())

### **DATES**

In [5]:
fire_dates = {
    'PineGulch': {
        'prefire_start': '2020-05-20',
        'prefire_end': '2020-06-18',
        'postfire_start': '2020-09-20',
        'postfire_end': '2020-10-28'
    },
    'GrizzlyCreek': {
        'prefire_start': '2020-05-20',
        'prefire_end': '2020-07-18',
        'postfire_start': '2020-09-06',
        'postfire_end': '2020-11-01'
    },
    'CherryCanyon':{
        'prefire_start': '2020-04-20',
        'prefire_end': '2020-05-15',
        'postfire_start': '2020-07-01',
        'postfire_end': '2020-08-15'
    },
    'SilverCreek': { #Done
        'prefire_start': '2021-06-01',
        'prefire_end': '2021-08-10',
        'postfire_start': '2019-05-10',
        'postfire_end': '2019-06-30'
    },
    'Decker': { # Kinda - not a huge fan of.... shadowing due to mountains east to west
        'prefire_start': '2019-05-30',
        'prefire_end': '2019-07-25',
       'postfire_start': '2020-5-10',
       'postfire_end': '2020-6-20'
    },
    'LakeChrist': { # done
        'prefire_start': '2017-08-01',
        'prefire_end': '2017-10-15',
        'postfire_start': '2018-09-15',
        'postfire_end': '2018-10-30'
    },
    'Palateau': { #Done
        'prefire_start': '2018-05-10',
        'prefire_end': '2018-06-20',
        'postfire_start': '2019-05-10',
        'postfire_end': '2019-06-20'
    }
}

fire_names = [
    'PineGulch',
    'GrizzlyCreek',
    'CherryCanyon',
    'Decker',
    'LakeChrist',
    'Palateau',
    'SilverCreek'
]

### **READ and SPLIT IMAGE**

In [ ]:
from sklearn.model_selection import train_test_split
import os
from osgeo import gdal
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from google.colab import drive
from skimage.filters import threshold_minimum
import numpy as np

def split_image_into_patches(image, patch_size, overlap_factor=2):
    height, width = image.shape
    patches = []
    stride = patch_size // overlap_factor
    for i in range(0, height - patch_size + 1, stride):
        for j in range(0, width - patch_size + 1, stride):
            patch = image[i:i+patch_size, j:j+patch_size]
            patches.append(patch)
    return np.array(patches)

#def split_image_into_patches_test(image, patch_size): No overlap for test
def split_image_into_patches_test(image, patch_size):
    height, width = image.shape
    patches = []
    stride = patch_size  #No overlap
    for i in range(0, height - patch_size + 1, stride):
        for j in range(0, width - patch_size + 1, stride):
            patch = image[i:i+patch_size, j:j+patch_size]
            patches.append(patch)
    return np.array(patches)

def read_geotiff(file_path):
    dataset = gdal.Open(file_path, gdal.GA_ReadOnly)
    if dataset is None:
        print(f"Failed to open file: {file_path}")
        return None
    if dataset.RasterCount < 1:
        print(f"No raster bands found in file: {file_path}")
        return None
    band = dataset.GetRasterBand(1)
    data = band.ReadAsArray()
    return data

drive.mount('/content/drive')

### **PATCHING**

In [8]:
#pip install scikit-image

In [ ]:
from skimage.filters import threshold_minimum
from scipy.ndimage import median_filter
from scipy.ndimage import uniform_filter
from scipy.ndimage import gaussian_filter
import cv2 as cv
import numpy as np

google_drive_path = "/content/drive/MyDrive/Cleaned_Fire_images_TIFF"
os.chdir(google_drive_path)

patch_size = 256
threshold = 85

def normalize_image(image, epsilon=1e-10):
    min_val = np.min(image)
    max_val = np.max(image)
    range_val = max_val - min_val
    if range_val == 0:
        return image
    else:
        range_safe = np.maximum(range_val, epsilon)
        return (image - min_val) / range_safe

def enhance_changes(image, threshold=0.5, factor=100, power=0.5):

    image_scaled = image * factor


    image_transformed = np.where(image_scaled < (threshold * factor),
                                 np.power(image_scaled, power),
                                 image_scaled)

    image_transformed /= factor

    return image_transformed


def extract_central_patch(image, size= (6 * 256)):
    height, width = image.shape
    top_left_x = (width - size) // 2
    top_left_y = (height - size) // 2
    return image[top_left_y:top_left_y+size, top_left_x:top_left_x+size]

import numpy as np

from collections import Counter

def classify_patch(patch):
    total_pixels = patch.size
    burned_pixels = np.sum(patch)

    burned_percentage = (burned_pixels / total_pixels) * 100

    if burned_percentage > 75:
        return "mostly burned"
    elif 50 < burned_percentage <= 75:
        return "half burned"
    elif 25 < burned_percentage <= 50:
        return "less burned"
    elif 10 < burned_percentage <= 25:
        return "vaguely burned"
    else:
        return "not burned"

def apply_denoising(image):
    # Parameters can be tuned based on image characteristics
    return cv.fastNlMeansDenoising(image, None, 30, 11, 31)


selected_images = ['PineGulch','SilverCreek'] #'PineGulch','LakeChrist',"Decker",
#selected_images = fire_names
dnbr_images = []
ground_truth_images = []
for fire_name in selected_images:

    file_path = f'S1/RATIO/S1_ratio_{fire_name}.tif'
    dnbr_image = read_geotiff(file_path)

    # Case 2&3 data Aug. and Box
    dnbr_image = extract_central_patch(dnbr_image)
    dnbr_image = enhance_changes(dnbr_image)

    dnbr_image = apply_denoising(dnbr_image.astype(np.uint8))
    dnbr_image = normalize_image(dnbr_image)

    dnbr_image_focal = dnbr_image

    #Pay attention to what S2 image you pull from
    binary_ground_truth_image = read_geotiff(f'S2/Resized_S2/TEST/resized_S2_{fire_name}_test.tif')
    binary_ground_truth_image1 = extract_central_patch(binary_ground_truth_image)
    binary_ground_truth_image2 = normalize_image(binary_ground_truth_image1)
    binary_ground_truth_image_threshold = (binary_ground_truth_image2 > 0.45).astype(np.uint8)

    #print(dnbr_image.shape)
    #print(binary_ground_truth_image_threshold.shape)

    fig, axes = plt.subplots(1, 2, figsize=(10, 5))

    axes[0].imshow(dnbr_image_focal, cmap='gray')
    axes[0].set_title('DNBR Image')
    axes[0].axis('off')

    axes[1].imshow(binary_ground_truth_image_threshold, cmap='gray')
    axes[1].set_title('Ground Truth Image')
    axes[1].axis('off')

    print(threshold)
    print(dnbr_image_focal[:5, :5])
    print(binary_ground_truth_image_threshold[:5, :5])

    # Split the dNBR image and ground truth image into patches with overlap
    dnbr_patches = split_image_into_patches(dnbr_image_focal, patch_size, overlap_factor=2)
    ground_truth_patches = split_image_into_patches(binary_ground_truth_image_threshold, patch_size, overlap_factor=2)


    classifications_truth = [classify_patch(patch) for patch in ground_truth_patches]
    distribution_truth = Counter(classifications_truth)
    print(f"{distribution_truth} - Ground Truth")

    classifications = [classify_patch(patch) for patch in dnbr_patches]
    distribution_b = Counter(classifications)
    print(f"{distribution_b} - dNBR")


    print(dnbr_patches.shape)
    print(ground_truth_patches.shape)

    dnbr_images.append(dnbr_patches)
    ground_truth_images.append(ground_truth_patches)

dnbr_images = np.concatenate(dnbr_images)
ground_truth_images = np.concatenate(ground_truth_images)

print(dnbr_images.shape)
print(ground_truth_images.shape)


DATA AUGMENTATION

In [ ]:
labels, values = zip(*distribution_truth.items())
indices = np.arange(len(labels))
width = 1

plt.bar(indices, values, width)
plt.xticks(indices, labels, rotation=45)
plt.ylabel('Number of Patches')
plt.title('Distribution of Burn Classifications')
plt.tight_layout()
plt.show()

labels, values = zip(*distribution_b.items())
indices = np.arange(len(labels))
width = 1

plt.bar(indices, values, width)
plt.xticks(indices, labels, rotation=45)
plt.ylabel('Number of Patches')
plt.title('Distribution of Burn Classifications')
plt.tight_layout()
plt.show()

In [ ]:
from imgaug import augmenters as iaa
import numpy as np

# Define your augmentation sequence
augmenter = iaa.Sequential([
    iaa.Fliplr(0.5),  # horizontal flips
    iaa.Flipud(0.5),  # vertical flips
    iaa.Affine(rotate=(0, 360))  # rotate between 0 and 90 degrees
])

def apply_augmentation_pairwise(dnbr_set, ground_truth_set, num_repeats=3):
    augmented_dnbr = []
    augmented_ground_truth = []

    for _ in range(num_repeats):  # Repeat the augmentation process num_repeats times
        for dnbr_img, gt_img in zip(dnbr_set, ground_truth_set):
            # Apply the same augmentation to both images
            aug_det = augmenter.to_deterministic()  # ensures the same augmentation is applied to both images
            augmented_dnbr.append(aug_det.augment_image(dnbr_img))
            augmented_ground_truth.append(aug_det.augment_image(gt_img))

    return np.array(augmented_dnbr), np.array(augmented_ground_truth)

# Assuming dnbr_images and ground_truth_images are defined elsewhere in your code
original_dnbr_tile_count = dnbr_images.shape[0]
original_ground_truth_tile_count = ground_truth_images.shape[0]

print("Number of original dNBR tiles:", original_dnbr_tile_count)
print("Number of original ground truth tiles:", original_ground_truth_tile_count)

# Apply augmentations, tripling the dataset size
augmented_dnbr_images, augmented_ground_truth_images = apply_augmentation_pairwise(dnbr_images, ground_truth_images, num_repeats=3)

# Print out the new distribution
print("Number of augmented dNBR tiles:", len(augmented_dnbr_images))
print("Number of augmented ground truth tiles:", len(augmented_ground_truth_images))


In [ ]:
c_dnbr_images = np.concatenate((dnbr_images, augmented_dnbr_images))
c_ground_truth_images = np.concatenate((ground_truth_images, augmented_ground_truth_images))

print("Total number of dNBR tiles (original + augmented):", c_dnbr_images.shape[0])
print("Total number of ground truth tiles (original + augmented):", c_ground_truth_images.shape[0])

In [ ]:
plt.figure(figsize=(6, 6))
plt.hist(c_dnbr_images.flatten(), bins=50, color='blue', alpha=0.7)
plt.title('DNBR Image Histogram')
plt.xlabel('Pixel Value')
plt.ylabel('Frequency')
plt.show()

# Create a new figure for the binary_ground_truth_image histogram
plt.figure(figsize=(6, 6))
plt.hist(c_ground_truth_images.flatten(), bins=50, color='green', alpha=0.7)
plt.title('S2 Raw Image Histogram')
plt.xlabel('Pixel Value')
plt.ylabel('Frequency')
plt.show()

### ***Looking at patches and effectivness ***

In [ ]:
import matplotlib.pyplot as plt

# patches
dnbr_patches_sample = c_dnbr_images[:10]
ground_truth_patches_sample = c_ground_truth_images[:10]

fig, axes = plt.subplots(10, 2, figsize=(10, 50))

for i in range(10):
    # Show dNBR patch
    ax = axes[i, 0]
    ax.imshow(dnbr_patches_sample[i], cmap='gray')
    ax.set_title(f'dNBR patch {i}')

    # Show ground truth patch
    ax = axes[i, 1]
    ax.imshow(ground_truth_patches_sample[i], cmap='gray')
    ax.set_title(f'Ground truth patch {i}')

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
threshold = 0.45

#dnbr_image = read_geotiff(file_path)

print("Minimum dNBR value:", np.min(dnbr_image))
print("Maximum dNBR value:", np.max(dnbr_image))
print("Mean (Average) dNBR value:", np.mean(dnbr_image))
print("Standard Deviation (SD) of dNBR:", np.std(dnbr_image))
print("Median dNBR value:", np.median(dnbr_image))
print("Sum of dNBR values:", np.sum(dnbr_image))
print("Variance of dNBR values:", np.var(dnbr_image))

#Histo
plt.hist(dnbr_image.flatten(), bins=50, color='c')
plt.title("Histogram of dNBR values")
plt.show()

#binary_ground_truth_image_threshold = (binary_ground_truth_image > threshold).astype(np.uint8)
binary_ground_truth_image_threshold

#Vis
plt.imshow(binary_ground_truth_image_threshold, cmap='gray')
plt.title("Binary Ground Truth Image")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from ipywidgets import widgets, interactive

def update(threshold):
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))

    # dNBR image thresholded
    dnbr_image_t = (dnbr_image > threshold).astype(np.uint8)
    ax = axes[0]
    ax.imshow(dnbr_image_t, cmap='gray')
    ax.set_title('dNBR Image Thresholded')

    # Ground truth thresholded
    binary_ground_truth_image_t = (binary_ground_truth_image > threshold).astype(np.uint8)
    ax = axes[1]
    ax.imshow(binary_ground_truth_image_t, cmap='gray')
    ax.set_title('Ground Truth Image Thresholded')

    plt.tight_layout()
    plt.show()

# Create interactive slider
threshold_slider = widgets.FloatSlider(
    value=0.5,
    min=0,
    max=1,
    step=0.05,
    description='Threshold:',
    continuous_update=False
)

# Display the widget
interactive(update, threshold=threshold_slider)


### **CNN**

In [ ]:
print(len(c_dnbr_images))
print(len(c_ground_truth_images))

print(len(dnbr_patches))
print(len(ground_truth_patches))

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, Input, Conv2DTranspose, concatenate
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Lambda, Input

from tensorflow.keras.models import load_model

#unet_model = create_unet_model()
#loaded_model = load_model('/content/drive/MyDrive/Cleaned_Fire_images_TIFF/Case1:BaseCase/unet_model_case1.5.30_RES.h5')
#unet_model = loaded_model
#combined_model  = loaded_model

In [ ]:
def learning_rate_schedule(epoch):
    if epoch < 28:
        return 0.00001  # first 30 epochs
    else:
        return 0.000001  # last 20 epochs

def create_unet_model(input_shape=(256,256, 1)):
    inputs = Input(shape=input_shape)

    # Encoder
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = BatchNormalization()(c1)
    p1 = MaxPooling2D((2, 2))(c1)
    p1 = Dropout(0.25)(p1)

    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = BatchNormalization()(c2)
    p2 = MaxPooling2D((2, 2))(c2)
    p2 = Dropout(0.25)(p2)

    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = BatchNormalization()(c3)
    p3 = MaxPooling2D((2, 2))(c3)
    p3 = Dropout(0.25)(p3)

    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = BatchNormalization()(c4)
    p4 = MaxPooling2D((2, 2))(c4)
    p4 = Dropout(0.25)(p4)

    # Bottom
    b = Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    b = BatchNormalization()(b)

    # Decoder
    u1 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(b)
    u1 = concatenate([u1, c4])
    u1 = Dropout(0.25)(u1)
    d1 = Conv2D(512, (3, 3), activation='relu', padding='same')(u1)
    d1 = BatchNormalization()(d1)

    u2 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(d1)
    u2 = concatenate([u2, c3])
    u2 = Dropout(0.25)(u2)
    d2 = Conv2D(256, (3, 3), activation='relu', padding='same')(u2)
    d2 = BatchNormalization()(d2)

    u3 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(d2)
    u3 = concatenate([u3, c2])
    u3 = Dropout(0.25)(u3)
    d3 = Conv2D(128, (3, 3), activation='relu', padding='same')(u3)
    d3 = BatchNormalization()(d3)

    u4 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(d3)
    u4 = concatenate([u4, c1])
    u4 = Dropout(0.25)(u4)
    d4 = Conv2D(64, (3, 3), activation='relu', padding='same')(u4)
    d4 = BatchNormalization()(d4)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(d4)

    model = Model(inputs=inputs, outputs=outputs)
   # model.compile(optimizer=Adam(learning_rate=0.00001), loss=BinaryCrossentropy(from_logits=False), metrics=['binary_accuracy'])
    return model

lr_scheduler = LearningRateScheduler(learning_rate_schedule)
unet_model = create_unet_model()

unet_model.compile(optimizer=Adam(learning_rate=0.00001),  # Start with initial learning rate
                   loss=BinaryCrossentropy(from_logits=False),
                   metrics=['binary_accuracy'])

# Train the model
X_train, X_test, y_train, y_test = train_test_split(c_dnbr_images, c_ground_truth_images, test_size=0.3)

X_train = X_train[..., np.newaxis]
X_test = X_test[..., np.newaxis]

batch_size = 32
epochs = 50
model_cnn = unet_model.fit(X_train, y_train,
                               batch_size,
                               epochs,
                               validation_data=(X_test, y_test),
                               callbacks=[lr_scheduler])


In [ ]:
# saving and loading the model
from tensorflow.keras.models import load_model

#unet_model.save('case5.h5')
#combined_model.save('case5.h5')
#loaded_model = load_model('.h5')
#unet_model = loaded_model


In [ ]:
unet_model.summary()
#combined_model.summary()
#tf.keras.utils.plot_model(model, "my_fashion_mnist_model.png", show_shapes=True)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

pd.DataFrame(model_cnn.history).plot(
    figsize=(8, 5), xlim=[0,50], ylim=[0, 1], grid=True, xlabel="Epoch",
    style=["r--", "r--.", "b-", "b-*"])
plt.legend(loc="lower left")  # extra code
#save_fig("keras_learning_curves_plot")  # extra code

save_path = 'keras_learning_curves_plot_CASE5.png'
plt.savefig(save_path, format='png')

plt.show()

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(unet_model, to_file='unet_model_5.png', show_shapes=True, show_layer_names=True)

### **ReCONSTRUCT IMAGES**

In [ ]:
def reconstruct_image_from_patches(patches, original_shape, patch_size, overlap_factor=2):
    height, width = original_shape
    reconstructed_image = np.zeros(original_shape)
    patch_count = np.zeros(original_shape)
    stride = patch_size // overlap_factor
    patch_idx = 0

    for i in range(0, height - patch_size + 1, stride):
        for j in range(0, width - patch_size + 1, stride):
            reconstructed_image[i:i+patch_size, j:j+patch_size] += patches[patch_idx]
            patch_count[i:i+patch_size, j:j+patch_size] += 1
            patch_idx += 1

    reconstructed_image /= patch_count
    return reconstructed_image

def reconstruct_image_from_patches_test(patches, original_shape, patch_size):
    height, width = original_shape
    reconstructed_image = np.zeros(original_shape)
    stride = patch_size  #No overlap
    patch_idx = 0
    for i in range(0, height - patch_size + 1, stride):
        for j in range(0, width - patch_size + 1, stride):
            reconstructed_image[i:i+patch_size, j:j+patch_size] = patches[patch_idx]
            patch_idx += 1

    return reconstructed_image

In [ ]:
dnbr_images_expanded = c_dnbr_images[..., np.newaxis]
predictions = unet_model.predict(dnbr_images_expanded)
#predictions = combined_model.predict(dnbr_images_expanded)
#Reconstruct the image from the patches
predictions = predictions[..., 0]
reconstructed_image = reconstruct_image_from_patches(predictions, dnbr_image.shape, patch_size, overlap_factor=2)

In [ ]:
#pip install rasterio

In [ ]:
from PIL import Image
from skimage import exposure

def save_image_as_jpeg(image_array, output_path):
    image_array = image_array * 500
    image_uint8 = Image.fromarray(image_array.clip(0, 255).astype(np.uint8))
    image_uint8.save(output_path)


import rasterio
def save_image_as_tif(image_path, save_path):
    with rasterio.open(image_path) as src:
        with rasterio.open(save_path, 'w', driver='GTiff',
                           width=src.width, height=src.height,
                           count=src.count, dtype=src.dtypes[0],
                           crs=src.crs, transform=src.transform) as dst:
            dst.write(src.read())

output_tif_path = 'Case5.tif'
output_jpeg_path = 'Case6.jpeg'

#save_image_as_tif(reconstructed_image, output_tif_path)
save_image_as_jpeg(reconstructed_image, output_jpeg_path)

### **RUNNING CNN**

In [ ]:
#https://docs.opencv.org/3.4/d5/d69/tutorial_py_non_local_means.html
import cv2 as cv

fire_name = fire_names[0]
new_image_path = f'S1/RATIO/S1_ratio_{fire_name}.tif'
new_image = read_geotiff(new_image_path)

# Apply the same preprocessing steps as done for the training images
new_image = read_geotiff(new_image_path)
preprocessed_new_image = new_image
preprocessed_new_image = enhance_changes(preprocessed_new_image)
preprocessed_new_image = apply_denoising(preprocessed_new_image.astype(np.uint8))
preprocessed_new_image = normalize_image(preprocessed_new_image)

# Split the preprocessed image into patches
patch_size = 256
patches = split_image_into_patches_test(preprocessed_new_image, patch_size)

# Apply the trained CNN model to the patches
patches_reshaped = patches.reshape(patches.shape[0], patches.shape[1], patches.shape[2], 1)
new_image_predictions = unet_model.predict(patches_reshaped)
new_image_predictions = new_image_predictions[..., 0]

# Reconstruct the image from the predicted patches
reconstructed_new_image = reconstruct_image_from_patches_test(new_image_predictions, new_image.shape, patch_size)

# Apply a threshold to create a binary mask of burned areas
threshold = 0.5 # Option to change model output againt preformance metrics
burned_area_mask = (reconstructed_new_image < threshold).astype(np.uint8)

# Save the output images
# output_tif_path = f'{fire_name}_Case5.{threshold}.tif'
output_jpeg_path = f'{fire_name}_Case5.{threshold}.jpeg'
save_image_as_jpeg(reconstructed_new_image, output_jpeg_path)

# Print statistics about the reconstructed image
print("Min pixel value:", np.min(reconstructed_new_image))
print("Max pixel value:", np.max(reconstructed_new_image))
print("Mean pixel value:", np.mean(reconstructed_new_image))
print("Standard Deviation:", np.std(reconstructed_new_image))

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(reconstructed_new_image.flatten(), bins=30, alpha=0.7)
plt.title('Histogram of Pixel Value Distributions')
plt.xlabel('Pixel Intensity')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

### **EVALUATIONS**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from matplotlib.colors import ListedColormap
import os
from osgeo import gdal


from google.colab import drive
drive.mount('/content/drive')
google_drive_path = "/content/drive/MyDrive/Cleaned_Fire_images_TIFF"
os.chdir(google_drive_path)

def calculate_metrics(y_true, y_pred):
    TP = np.sum((y_true == 1) & (y_pred == 1))
    FP = np.sum((y_true == 0) & (y_pred == 1))
    TN = np.sum((y_true == 0) & (y_pred == 0))
    FN = np.sum((y_true == 1) & (y_pred == 0))

    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1_score = 2 * (precision * recall) / (precision + recall)
    iou = TP / (TP + FP + FN)
    rel_bias = (TP + FP) / (TP + FN) - 1

    return precision, recall, f1_score, iou, rel_bias

def trim_images_to_smallest(image1, image2):
    shape1 = np.array(image1.shape)
    shape2 = np.array(image2.shape)
    min_shape = np.minimum(shape1, shape2)
    return image1[:min_shape[0], :min_shape[1]], image2[:min_shape[0], :min_shape[1]]

def trim_to_ground_truth(ground_truth, predicted):
    """Crop the predicted image to match the shape of the ground truth."""
    return ground_truth, predicted[:ground_truth.shape[0], :ground_truth.shape[1]]

def center_crop_to_ground_truth(ground_truth, predicted):
    """Center-crop the predicted image to match the size of the ground truth."""
    gt_rows, gt_cols = ground_truth.shape
    pred_rows, pred_cols = predicted.shape

    center_row, center_col = pred_rows // 2, pred_cols // 2
    half_gt_rows, half_gt_cols = gt_rows // 2, gt_cols // 2

    start_row = max(center_row - half_gt_rows, 0)
    end_row = start_row + gt_rows
    start_col = max(center_col - half_gt_cols, 0)
    end_col = start_col + gt_cols

    return ground_truth, predicted[start_row:end_row, start_col:end_col]

def normalize_image(image, epsilon=1e-10):
    min_val = np.min(image)
    max_val = np.max(image)
    range_val = max_val - min_val
    if range_val == 0:
        return image
    else:
        range_safe = np.maximum(range_val, epsilon)
        return (image - min_val) / range_safe

def read_geotiff(file_path):
    dataset = gdal.Open(file_path, gdal.GA_ReadOnly)
    if dataset is None:
        print(f"Failed to open file: {file_path}")
        return None
    if dataset.RasterCount < 1:
        print(f"No raster bands found in file: {file_path}")
        return None
    band = dataset.GetRasterBand(1)
    data = band.ReadAsArray()
    return data

fires = ['CherryCanyon', 'GrizzlyCreek','PineGulch', 'SilverCreek', "Decker", "LakeChrist",'Palateau']
#fires = ['Palateau']


for fire in fires:
    dnbr1 = read_geotiff(f'{fire}.tif')
    dnbr = normalize_image(dnbr1)

    ground_truth = read_geotiff(f'{fire}.tif')
    ground_truth1 = normalize_image(ground_truth)
    ground_truth_binary = (ground_truth1 > 0.45).astype(np.uint8)

    model_output1 = cv2.imread(f'Case5{fire}.jpeg', cv2.IMREAD_GRAYSCALE)

    model_output1
    model_output_norm = model_output1 / 255.0
    model_output = np.round(model_output_norm).astype(np.uint8)


    #Discrepancy map
    discrepancy_rgb = np.zeros((*ground_truth_binary.shape, 3), dtype=np.uint8)

    discrepancy_rgb[np.logical_and(ground_truth_binary == 1, model_output == 1)] = [255, 255, 255]  # white (true unburned)
    discrepancy_rgb[np.logical_and(ground_truth_binary == 0, model_output == 0)] = [0, 255, 0]      # green (true burned)
    discrepancy_rgb[np.logical_and(ground_truth_binary == 1, model_output == 0)] = [0, 0, 255]      # blue (false positives)
    discrepancy_rgb[np.logical_and(ground_truth_binary == 0, model_output == 1)] = [255, 0, 0]      # red (false negatives)


    #Vis
    fig, axes = plt.subplots(1, 4, figsize=(20, 5))

    axes[0].imshow(dnbr, cmap='gray')
    axes[0].set_title(f'{fire} - dNBR')
    axes[0].axis('off')

    axes[1].imshow(ground_truth_binary, cmap='gray')
    axes[1].set_title(f'{fire} - Ground Truth')
    axes[1].axis('off')

    axes[2].imshow(model_output, cmap='gray')
    axes[2].set_title(f'{fire} - Model Output')
    axes[2].axis('off')

    im = axes[3].imshow(discrepancy_rgb)
    axes[3].set_title(f'{fire} - Discrepancy Map')
    axes[3].axis('off')

    #Adding custom legend
    from matplotlib.lines import Line2D
    legend_elements = [Line2D([0], [0], marker='o', color='w', markerfacecolor='white', markersize=10, label='Correct Unburned'),
                       Line2D([0], [0], marker='o', color='w', markerfacecolor='green', markersize=10, label='Correct Burned'),
                       Line2D([0], [0], marker='o', color='w', markerfacecolor='blue', markersize=10, label='False Burend'),
                       Line2D([0], [0], marker='o', color='w', markerfacecolor='red', markersize=10, label='False Unburned')]
    axes[3].legend(handles=legend_elements, loc='upper right')

    plt.tight_layout()
    plt.show()

    #print("Ground Truth Min:", ground_truth_binary.min(), "Max:", ground_truth_binary.max())
    #print("Model Output Min:", model_output.min(), "Max:", model_output.max())
    #print(results_table)



In [ ]:
import pandas as pd
import numpy as np
import cv2

metrics_dict = {
    'Fire Name': [],
    'Threshold': [],
    'Precision': [],
    'Recall': [],
    'F1 Score': [],
    'IoU': [],
    'Relative Bias': []
}

fire_names = [
   'PineGulch',
   'GrizzlyCreek',
   'CherryCanyon',
    'Decker',
    'LakeChrist',
    'Palateau',
    'SilverCreek'
]

thresholds = [9] # Possability to add more thresholds depending on images created for evaluation in previous steps.

for fire_name in fire_names:
    for threshold in thresholds:
        ground_truth = read_geotiff(f'S2/Resized_S2/TEST/resized_S2_{fire_name}_test.tif')
        ground_truth1 = normalize_image(ground_truth)
        ground_truth_binary = (ground_truth1 > 0.45).astype(np.uint8)

        model_output1 = cv2.imread(f'Case5{fire}.jpeg', cv2.IMREAD_GRAYSCALE)
        print(fire_name, threshold)
        model_output_norm = model_output1 / 255.0
        model_output_binary = (model_output_norm > 0.5).astype(np.uint8)

        #print (model_output_norm)
        #print (model_output_binary)

        ground_truth_binary, model_output_binary = trim_images_to_smallest(ground_truth_binary, model_output_norm)

        precision, recall, f1_score, iou, dice, rel_bias = calculate_metrics(ground_truth_binary, model_output_binary)

        metrics_dict['Fire Name'].append(fire_name)
        metrics_dict['Threshold'].append(threshold)
        metrics_dict['Precision'].append(precision)
        metrics_dict['Recall'].append(recall)
        metrics_dict['F1 Score'].append(f1_score)
        metrics_dict['IoU'].append(iou)
        metrics_dict['Relative Bias'].append(rel_bias)

metrics_df = pd.DataFrame(metrics_dict)
print(metrics_df)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Data from table
data = {
    'Fire Name': ['PineGulch', 'GrizzlyCreek', 'CherryCanyon', 'Decker', 'LakeChrist', 'Palateau', 'SilverCreek'],
    'F1 Score': [0.6664, 0.6624, 0.7307, 0.7619, 0.7178, 0.7554, 0.4765],
    'IoU': [0.4997, 0.4952, 0.5756, 0.6154, 0.5598, 0.6069, 0.3128],
    'Relative Bias': [-0.0869, -0.2065, -0.4211, -0.1373, -0.0312, -0.3386, 0.3242]
}

df = pd.DataFrame(data)

fig, axes = plt.subplots(3, 1, figsize=(10, 15))

# Plot F1 Score
df.plot(x='Fire Name', y='F1 Score', kind='bar', ax=axes[0], color='skyblue', legend=False)
axes[0].set_title('F1 Score')
axes[0].set_ylim(0, 1)  # Setting the limit for y axis
axes[0].set_xticklabels(df['Fire Name'], rotation=45, ha='right')  # Rotate x-axis labels by 45 degrees

# Plot IoU
df.plot(x='Fire Name', y='IoU', kind='bar', ax=axes[1], color='#90EE90', legend=False)
axes[1].set_title('IoU')
axes[1].set_ylim(0, 1)  # Setting the limit for y axis
axes[1].set_xticklabels(df['Fire Name'], rotation=45, ha='right')  # Rotate x-axis labels by 45 degrees

# Plot Relative Bias
df.plot(x='Fire Name', y='Relative Bias', kind='bar', ax=axes[2], color='salmon', legend=False)
axes[2].set_title('Relative Bias')
axes[2].set_ylim(-0.5, 0.5)  # Setting the limit for y axis
axes[2].set_xticklabels(df['Fire Name'], rotation=45, ha='right')  # Rotate x-axis labels by 45 degrees

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Data from Case 1
data = {
    'Fire Name': ['CherryCanyon', 'GrizzlyCreek', 'PineGulch', 'SilverCreek',
                  'Decker', 'LakeChrist', 'Palateau'],
    'F1 Score': [0.8802, 0.7858, 0.7925, 0.5342, 0.9131, 0.8384, 0.8244],
    'IoU': [0.7861, 0.6348, 0.6263, 0.3654, 0.8446, 0.7292, 0.6959],
    'Relative Bias': [-0.1893, 0.1113, 0.2892, 0.8602, 0.1790, 0.3695, -0.0673]
}

df = pd.DataFrame(data)

# Set up the matplotlib figure
fig, axes = plt.subplots(3, 1, figsize=(10, 15))

# Plot F1 Score
df.plot(x='Fire Name', y='F1 Score', kind='bar', ax=axes[0], color='skyblue', legend=False)
axes[0].set_title('F1 Score')
axes[0].set_ylim(0, 1)  # Setting the limit for y axis
axes[0].set_xticklabels(df['Fire Name'], rotation=45, ha='right')  # Rotate x-axis labels by 45 degrees

# Plot IoU
df.plot(x='Fire Name', y='IoU', kind='bar', ax=axes[1], color='#90EE90', legend=False)
axes[1].set_title('IoU')
axes[1].set_ylim(0, 1)  # Setting the limit for y axis
axes[1].set_xticklabels(df['Fire Name'], rotation=45, ha='right')  # Rotate x-axis labels by 45 degrees

# Plot Relative Bias
df.plot(x='Fire Name', y='Relative Bias', kind='bar', ax=axes[2], color='salmon', legend=False)
axes[2].set_title('Relative Bias')
axes[2].set_ylim(-1, 1)  # Adjusted limit to accommodate the full range of your new data
axes[2].set_xticklabels(df['Fire Name'], rotation=45, ha='right')  # Rotate x-axis labels by 45 degrees

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Data from Case 2&3
data = {
    'Fire Name': ['CherryCanyon', 'GrizzlyCreek', 'PineGulch', 'SilverCreek',
                  'Decker', 'LakeChrist', 'Palateau'],
    'F1 Score': [0.6038, 0.5661, 0.6020, 0.4785, 0.6240, 0.6337, 0.6458],
    'IoU': [0.4324, 0.3948, 0.4306, 0.3145, 0.4534, 0.4638, 0.4769],
    'Relative Bias': [-0.5647, -0.4034, -0.3135, -0.0045, -0.3450, -0.2761, -0.5028]
}

df = pd.DataFrame(data)

# Set up the matplotlib figure
fig, axes = plt.subplots(3, 1, figsize=(10, 15))

# Plot F1 Score
df.plot(x='Fire Name', y='F1 Score', kind='bar', ax=axes[0], color='skyblue', legend=False)
axes[0].set_title('F1 Score')
axes[0].set_ylim(0, 1)  # Setting the limit for y axis
axes[0].set_xticklabels(df['Fire Name'], rotation=45, ha='right')  # Rotate x-axis labels by 45 degrees

# Plot IoU
df.plot(x='Fire Name', y='IoU', kind='bar', ax=axes[1], color='#90EE90', legend=False)
axes[1].set_title('IoU')
axes[1].set_ylim(0, 1)  # Setting the limit for y axis
axes[1].set_xticklabels(df['Fire Name'], rotation=45, ha='right')  # Rotate x-axis labels by 45 degrees

# Plot Relative Bias
df.plot(x='Fire Name', y='Relative Bias', kind='bar', ax=axes[2], color='salmon', legend=False)
axes[2].set_title('Relative Bias')
axes[2].set_ylim(-1, 1)  # Adjusted limit to accommodate the range of bias data
axes[2].set_xticklabels(df['Fire Name'], rotation=45, ha='right')  # Rotate x-axis labels by 45 degrees

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Data from Case 4
data = {
    'Fire Name': ['CherryCanyon', 'GrizzlyCreek', 'PineGulch', 'SilverCreek',
                  'Decker', 'LakeChrist', 'Palateau'],
    'F1 Score': [0.9453, 0.7117, 0.8594, 0.7658, 0.7716, 0.5734, 0.7696],
    'IoU': [0.8963, 0.5525, 0.7535, 0.6205, 0.6282, 0.4019, 0.6254],
    'Relative Bias': [-0.0762, 0.2023, 0.1118, 0.0538, -0.2590, -0.4774, -0.3187]
}

df = pd.DataFrame(data)

# Set up the matplotlib figure
fig, axes = plt.subplots(3, 1, figsize=(10, 15))

# Plot F1 Score
df.plot(x='Fire Name', y='F1 Score', kind='bar', ax=axes[0], color='skyblue', legend=False)
axes[0].set_title('F1 Score')
axes[0].set_ylim(0, 1)  # Setting the limit for y axis
axes[0].set_xticklabels(df['Fire Name'], rotation=45, ha='right')  # Rotate x-axis labels by 45 degrees

# Plot IoU
df.plot(x='Fire Name', y='IoU', kind='bar', ax=axes[1], color='#90EE90', legend=False)
axes[1].set_title('IoU')
axes[1].set_ylim(0, 1)  # Setting the limit for y axis
axes[1].set_xticklabels(df['Fire Name'], rotation=45, ha='right')  # Rotate x-axis labels by 45 degrees

# Plot Relative Bias
df.plot(x='Fire Name', y='Relative Bias', kind='bar', ax=axes[2], color='salmon', legend=False)
axes[2].set_title('Relative Bias')
axes[2].set_ylim(-0.5, 0.5)  # Adjusted limit to accommodate the range of bias data
axes[2].set_xticklabels(df['Fire Name'], rotation=45, ha='right')  # Rotate x-axis labels by 45 degrees


plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Data Setup
fire_names = ['CherryCanyon', 'GrizzlyCreek', 'PineGulch', 'SilverCreek',
              'Decker', 'LakeChrist', 'Palateau']
cases = ['Case 1', 'Case 2&3', 'Case 4', 'Case 5']

# IoU data for each case and fire
iou_data = {
    'Fire Name': fire_names,
    'Case 1': [0.7861, 0.6348, 0.6263, 0.3654, 0.8446, 0.7292, 0.6959],
    'Case 2&3': [0.4324, 0.3948, 0.4306, 0.3145, 0.4534, 0.4638, 0.4769],
    'Case 4': [0.8963, 0.5525, 0.7535, 0.6205, 0.6282, 0.4019, 0.6254],
    'Case 5': [0.5756, 0.4952, 0.4997, 0.3128, 0.6154, 0.5598, 0.6069]
}

# Relative Bias data for each case and fire
relative_bias_data = {
    'Fire Name': fire_names,
    'Case 1': [-0.1893, 0.1113, 0.2892, 0.8602, 0.1790, 0.3695, -0.0673],
    'Case 2&3': [-0.5647, -0.4034, -0.3135, -0.0045, -0.3450, -0.2761, -0.5028],
    'Case 4': [-0.0762, 0.2023, 0.1118, 0.0538, -0.2590, -0.4774, -0.3187],
    'Case 5': [-0.4211, -0.2065, -0.0869, 0.3242, -0.1373, -0.0312, -0.3386]
}

# F1 Score data for each case and fire
f1_score_data = {
    'Fire Name': fire_names,
    'Case 1': [0.8802, 0.7858, 0.7925, 0.5342, 0.9131, 0.8384, 0.8244],
    'Case 2&3': [0.6038, 0.5661, 0.6020, 0.4785, 0.6240, 0.6337, 0.6458],
    'Case 4': [0.9453, 0.7117, 0.8594, 0.7658, 0.7716, 0.5734, 0.7696],
    'Case 5': [0.7307, 0.6624, 0.6664, 0.4765, 0.7619, 0.7178, 0.7554]
}

# Creating DataFrames
iou_df = pd.DataFrame(iou_data)
relative_bias_df = pd.DataFrame(relative_bias_data)
f1_score_df = pd.DataFrame(f1_score_data)

# plot and save figure
def plot_and_save(df, metric_name, filename):
    plt.figure(figsize=(10, 6))
    for index, row in df.iterrows():
        plt.plot(cases, row[1:], marker='o', label=row['Fire Name'])
    plt.title(f'{metric_name} Scores Across Cases')
    plt.xlabel('Case')
    plt.ylabel(metric_name)
    plt.legend(title='Fire Name')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f"{filename}.png")
    plt.close()

# Plotting and saving each graph
plot_and_save(iou_df, 'IoU', 'iou_scores')
plot_and_save(relative_bias_df, 'Relative Bias', 'relative_bias_scores')
plot_and_save(f1_score_df, 'F1 Score', 'f1_scores')